In [3]:
import pandas as pd

# Read the Excel file, skipping the first two rows
df = pd.read_excel('list2_2023.xlsx', skiprows=2)

df['FIPS State Code'] = df['FIPS State Code'].fillna(0).astype(int).astype(str).str.zfill(2)
df['FIPS Place Code'] = df['FIPS Place Code'].fillna(0).astype(int).astype(str).str.zfill(3)

# Save to CSV
output_filename = 'list2_2023.csv'
df.to_csv(output_filename, index=False)

print(f"Successfully converted list2_2023.xlsx to {output_filename}")
print(f"Removed first 2 rows from the original file")
print(f"Output file has {len(df)} rows and {len(df.columns)} columns")

Successfully converted list2_2023.xlsx to list2_2023.csv
Removed first 2 rows from the original file
Output file has 1297 rows and 6 columns


In [4]:
# Read the CSV files
list_df = pd.read_csv('list2_2023.csv')
zips_df = pd.read_csv('ZIPS_BY_POP.csv')

# Create a combined FIPS code from list2_2023
# FIPS State Code: zfill to 2 characters
# FIPS Place Code: zfill to 3 characters  
# Combined: 5-digit FIPS code
list_df['fips_state_str'] = list_df['FIPS State Code'].fillna(0).astype(int).astype(str).str.zfill(2)
list_df['fips_place_str'] = list_df['FIPS Place Code'].fillna(0).astype(int).astype(str).str.zfill(3)
list_df['fips_combined'] = list_df['fips_state_str'] + list_df['fips_place_str']

# Ensure the fips column in zips_df is string type for joining
zips_df['fips'] = zips_df['fips'].astype(str).str.zfill(5)

# Show some examples of the FIPS codes before joining
print("Sample FIPS codes from list2_2023:")
print(list_df[['CBSA Code', 'Principal City Name', 'FIPS State Code', 'FIPS Place Code', 'fips_combined']].head(10))
print("\nSample FIPS codes from ZIPS_BY_POP:")
print(zips_df[['zip', 'fips', 'state', 'county']].head(10))

# Perform the join
merged_df = pd.merge(
    list_df,
    zips_df,
    left_on='fips_combined',
    right_on='fips',
    how='left'
)

# Save the merged dataframe
output_filename = 'list2023_with_zips.csv'
merged_df.to_csv(output_filename, index=False)

print(f"\n{'='*60}")
print(f"Join completed!")
print(f"Original list2_2023 rows: {len(list_df)}")
print(f"ZIPS_BY_POP rows: {len(zips_df)}")
print(f"Merged rows: {len(merged_df)}")
print(f"Rows with matching FIPS: {merged_df['zip'].notna().sum()}")
print(f"Rows without matches: {merged_df['zip'].isna().sum()}")
print(f"Output saved to: {output_filename}")

# Show some matched results
print(f"\n{'='*60}")
print("Sample of matched results:")
matched = merged_df[merged_df['zip'].notna()].head(10)
print(matched[['CBSA Code', 'Principal City Name', 'fips_combined', 'zip', 'county', 'population']])

# Show some unmatched results
print(f"\n{'='*60}")
print("Sample of unmatched results (no zip found):")
unmatched = merged_df[merged_df['zip'].isna()].head(10)
print(unmatched[['CBSA Code', 'Principal City Name', 'FIPS State Code', 'FIPS Place Code', 'fips_combined']])

Sample FIPS codes from list2_2023:
  CBSA Code Principal City Name  FIPS State Code  FIPS Place Code  \
0     10100            Aberdeen               46              100   
1     10140            Aberdeen               53              100   
2     10180             Abilene               48             1000   
3     10220                 Ada               40              200   
4     10300              Adrian               26              440   
5     10380           Aguadilla               72              745   
6     10420               Akron               39             1000   
7     10460          Alamogordo               35             1780   
8     10480             Alamosa                8             1090   
9     10500              Albany               13             1052   

  fips_combined  
0         46100  
1         53100  
2        481000  
3         40200  
4         26440  
5         72745  
6        391000  
7        351780  
8        081090  
9        131052  

Sample

In [5]:
# Read the MSA_to_FIPS.csv
msa_df = pd.read_csv('MSA_to_FIPS.csv')

# Ensure the msa column is string type for merging
list_df['CBSA Code'] = list_df['CBSA Code'].astype(str)
msa_df['msa'] = msa_df['msa'].astype(str)

# Merge the dataframes on CBSA Code = msa
merged_df = pd.merge(
    list_df,
    msa_df,
    left_on='CBSA Code',
    right_on='msa',
    how='left'
)

# Save the merged dataframe
output_filename = 'merged_msa_data.csv'
merged_df.to_csv(output_filename, index=False)

print(f"Successfully merged the data")
print(f"Original list2_2023 rows: {len(list_df)}")
print(f"MSA_to_FIPS rows: {len(msa_df)}")
print(f"Merged rows: {len(merged_df)}")
print(f"Output saved to: {output_filename}")
print(f"\nFirst few rows of merged data:")
print(merged_df.head())



Successfully merged the data
Original list2_2023 rows: 1297
MSA_to_FIPS rows: 1845
Merged rows: 3766
Output saved to: merged_msa_data.csv

First few rows of merged data:
  CBSA Code    CBSA Title Metropolitan/Micropolitan Statistical Area  \
0     10100  Aberdeen, SD              Micropolitan Statistical Area   
1     10100  Aberdeen, SD              Micropolitan Statistical Area   
2     10140  Aberdeen, WA              Micropolitan Statistical Area   
3     10180   Abilene, TX              Metropolitan Statistical Area   
4     10180   Abilene, TX              Metropolitan Statistical Area   

  Principal City Name  FIPS State Code  FIPS Place Code fips_state_str  \
0            Aberdeen               46              100             46   
1            Aberdeen               46              100             46   
2            Aberdeen               53              100             53   
3             Abilene               48             1000             48   
4             Abilene      